In [ ]:
import numpy as np

# Para tratamiento y e/s de datos
import pandas as pd

# Gráficos de datos
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from fbprophet import Prophet
# from fbprophet.deagnostics import cross_validation

# Forecasting Demanda Energía (Prophet)

In [ ]:
# Importo el archivos de datos de consumo de energia en la zona este de EE.UU.
df = pd.read_csv(r'\kaggle_forecasting_EP\PJME_hourly.csv')
df.head()

<b>Target_value: "PJME_MW"</b>

In [ ]:
#Convierto el index en DateTimeIndex
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.sort_values(by=['Datetime'], axis = 0, ascending = True, inplace = True)
df.reset_index(inplace = True, drop = True)

# La variable objetivo (y) se renombra a: demand_in_MW
df.rename(columns={'PJME_MW':'demand_in_MW'}, inplace = True)

df.head()

In [ ]:
df.shape

## Limpieza de datos

### Eliminación de datos duplicados

In [ ]:
# De datos duplicados, solo se mantiene la medición más reciente. 
df.drop_duplicates(subset = 'Datetime', keep = 'last', inplace = True)
df.shape

### Tratamiento de espacios vacios para un grupo de datos continuos

In [ ]:
df_2 = df.set_index('Datetime')
# print(f'df.index.freq is set to: {df.index.freq}')
df_2

In [ ]:
print(f'df_2.index.freq is set to: {df_2.index.freq}')

<i>
Tener un dataset con frecuencia en "None" indica 
que existen datos que perdidos (missing). <br>
Para verificar lo dicho, podemos comparar con un rango de datos
custom e ininterrumpido
</i>

In [ ]:
# Custom range
data_range = pd.date_range(start = min(df_2.index),
                          end = max(df_2.index),
                          freq = 'H') 

data_range #NO ES UN DATAFRAME, ES UN DATETIMEINDEX (un index basicamente.) 
#freq = 'H' indica frecuencia por hora.
#Explicación: genero un dataframe con una frecuencia horaria desde el valor minimo del index (datetime)
#del dataframe original, y con el valor máximo del index. Con esto lo que obtengo es TODO EL CALENDARIO
#sin datos perdidos. 
#Al hacer mas adelante la diferencia entre ambos dataframe, voy a obtener los "días perdidos" del dataframe original. 
# https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases

In [ ]:
print(f'La diferencia de longitud entre el rango customizado de datos y nuestro dataset es {(len(data_range)-len(df_2))}:')
print(data_range.difference(df_2.index))
#la diferencia entre ambos df indica la cantidad de valores perdidos en el df_original

<i>Re-indexamos los valores nuevos al dataset</i>

In [ ]:
# El siguiente comando adjunta los datos "datetime" perdidos (missing) al dataset original
# pero va a generar valores NaN para la variable Target (Demand_in_MW)

df_3 = df_2.reindex(data_range)

# Llenamos estos valores blancos con valores que se encuentran en una curva lineal entre 
# puntos de datos existentes
df_3['demand_in_MW'].interpolate(method='linear', inplace=True)

# Con la interpolación se tiene un datetime (set de hora y dias) continuo
print(f'La df.index.freq ahora es: {df_3.index.freq}, indicando que ya no tenemos valores perdidos')

In [ ]:
df_3.head(5)

### Extraemos características de la variable Tiempo

<i>
Podemos dividir la columna de Datetime en sus diferentes componentes. <br>
Esto nos permite encontrar patrones para diferentes grupos.
</i>

In [ ]:
df_3['dow'] = df_3.index.day_of_week
df_3['doy'] = df_3.index.day_of_year
df_3['year'] = df_3.index.year
df_3['month'] = df_3.index.month
df_3['quarter'] = df_3.index.quarter
df_3['hour'] = df_3.index.hour
df_3['weekday'] = df_3.index.day_name()
df_3['woy'] = df_3.index.isocalendar().week #week of year
df_3['dom'] = df_3.index.day # Day of Month
df_3['date'] = df_3.index.date 

# número de estación del año
df_3['season'] = df_3['month'].apply(lambda month_number: (month_number%12 + 3)//3) 
# el operador aritmético // solo devuelve a parte entera de la división.

In [ ]:
df_3.index.year

## EDA

### Graficando el consumo de energía a lo largo del tiempo

In [ ]:
#Plotyle no permite acceso directo a los index del df. ?????
df_3['date_and_time'] = df_3.index 

#Plotting
fig = px.line(df_3, x=['date_and_time'], y='demand_in_MW', title=f'Demanda MW por tiempo [{min(df_3.year)} - {max(df_3.year)}]')
fig.update_traces(line=dict(width=0.05))
fig.update_layout(xaxis_title='Date & Time', yaxis_title='Demanda Energía [MW]')
fig.show()

Estudiando la gráfica se observa un comportamiento con patron en temporadas (estación del año). 

### Patrones de fecha y hora

Podemos usar nuestras funciones de fecha y hora extraídas previamente <br>
para ver si surgen patrones recurrentes de los datos agregados. <br>
Tomemos, por ejemplo, la demanda de energía a lo largo del día para cada día de la semana:

In [ ]:
df_3.columns

In [ ]:
# Dataframe definido para reflejar el consumo por hora en la semana, usando la mediana de energia. 
patron_1 = df_3.groupby(['hour', 'weekday'], as_index=False).agg({'demand_in_MW':'median'})
patron_1

In [ ]:
fig = px.line(patron_1, 
              x = 'hour',
              y = 'demand_in_MW', 
              color='weekday', 
              title='Mediana de consumo de energia por hs por día de semana ')

fig.update_layout(xaxis_title='Hour', yaxis_title='Energy Demand[MW]')

fig.show()

In [ ]:
# Dataframe definido para graficar el consumo horario por temporada del año. Mediana de la energía. 
patron_2 = df_3.groupby(['hour', 'season'], as_index=False).agg({'demand_in_MW':'median'})

In [ ]:
fig_2 = px.line(patron_2, 
                x = 'hour',
                y = 'demand_in_MW', 
                color='season', 
                title='Mediana de consumo de energia por hs por estación')

fig_2.update_layout(xaxis_title='Hour', yaxis_title='Energy Demand[MW]')

fig_2.show()

## Descompoción de la serie de tiempo

Los puntos que representan datos a lo largo de una serie de tiempo pueden ser interesantes <br>
en cuanto sus patrones se complementes con tendencias de subida/bajada y/o estacionalidad. <br>
Según la info adquirida en el EDA esto parece ser así.

In [ ]:
print(f'El primer punto de medicion fecha/hs es: {min(df_3.index)}')
print(f'El último punto de medicion fecha/hs es: {max(df_3.index)}')

In [ ]:
# Dataframe de recort
CUTOFF_DATE = pd.to_datetime('2017-08-01')
TIME_DELTA = pd.DateOffset(years=8)

# Separo df p/ test y df p/ train
train = df_3.loc[(df_3.index < CUTOFF_DATE) & (df_3.index >= CUTOFF_DATE - TIME_DELTA)].copy()
test = df_3.loc[df_3.index >= CUTOFF_DATE].copy()

In [ ]:
#Se permite recortar varias fechas porque:
#1- El comportamiento es constante en el tiempo.
#2- Alivia la carga de procesamiento en la PC.
print(f'Training shape: {train.shape}\n Testing shape: {test.shape}\n')
print(f'Las fechas de entrenamiento son: {min(train.index)} & {max(train.index)}')
print(f'Las fechas de test son: {min(test.index)} & {max(test.index)}')

# Prophet

Es un modelo de pronóstico de series de tiempo, diseñado para manejar las características comunes
en las series de tiempo implementadas hoy en día. <br>
La idea del modelo Prophet es ser accesible y ajustable sin necesitar tener conocimientos de lo que pasa
detrás del telón respecto al funcionamiento matemático de la serie de tiempo. <br>
Tecnicamente hablando, es una serie de tiempo descompuesta en tres términos:
<i>y(t) = g(t)+s(t)+h(t)+et</i>
<ul>
<li>g(t): trend
    <blockquote> 
        Función de tendencia que modela cambios no-periodicos en los valores de la serie de tiempo.
    </blockquote>
    </li> 
<li>s(t): seasonality
    <blockquote>   
        Función que representa cambios periodicos. 
    </blockquote>
    </li> 
<li>h(t): holidays
    <blockquote>  
        Función que representa los efectos de los días de vacaciones/feriados/findes.
    </blockquote>
    </li>
<li>et: Término de error. 
    <blockquote>  
        Representa cualquier cambio idiosincracico (herencia). Se supone normalmente distribuido. 
    </blockquote>
    </li>
</ul>

Docs Oficiales (muy utiles): __[PROPHET_DOCS](https://facebook.github.io/prophet/docs/quick_start.html)__<BR>
Teoría: __[Forecasting at Scale(pdf)](https://www.kaggle.com/robinteuwens/forecasting-energy-demand/notebook)__ <br>
Practica: __[Forecasting con Prophet](https://nextjournal.com/eric-brown/forecasting-with-prophet)__ 

In [ ]:
# Formato de datos para el modelo de Prophet 
train_prophet = train[['demand_in_MW']].reset_index().rename(columns = {'index': 'ds', 'demand_in_MW': 'y'})
test_prophet = test[['demand_in_MW']].reset_index().rename(columns = {'index': 'ds', 'demand_in_MW': 'y'})
test_prophet.info()

<b>Los dos bloques que siguen me sirven para agregar fechas extras al DataFrame</b>

In [ ]:
from datetime import datetime, timedelta

inicio = datetime(2018,8,3) 
fin = datetime(2020,1,1)

lista_fechas = [(inicio + timedelta(hours=h)) for h in range(((fin-inicio).days + 1)*24)]

print(f'Se agregarán: {(max(lista_fechas)-min(lista_fechas))*24}')
print(min(lista_fechas))
print(max(lista_fechas))

In [ ]:
fechas_extras = pd.DataFrame(lista_fechas).rename(columns = {0: 'ds'})
fechas_extras.head()

In [ ]:
test_prophet_2 = pd.concat([test_prophet, fechas_extras])
test_prophet_2.reset_index(inplace = True, drop = True)
# De datos duplicados, solo se mantiene la medición más reciente. 
test_prophet_2.drop_duplicates(subset = 'ds', keep = 'last', inplace = True)
test_prophet_2.info()

## Conditional Seasonalities

<blockquote>
In some instances the seasonality may depend on other factors, such as a weekly seasonal pattern that is different during the summer than it is during the rest of the year, or a daily seasonal pattern that is different on weekends vs. on weekdays. These types of seasonalities can be modeled using conditional seasonalities.
</blockquote>

Del EDA podemos observar que la variación diaria en estaciones es mayor en Verano e Invierno (obviamente). <br>
Destripemos los patrones de los datos para tener en cuenta la interdependencia de estas variables.

In [ ]:
# Condiciones
def is_spring(ds): 
    date = pd.to_datetime(ds)    
    return (date.month >= 3) & (date.month <=5)

def is_summer(ds): 
    date = pd.to_datetime(ds)
    return (date.month >= 6) & (date.month <=8)

def is_autumn(ds): 
    date = pd.to_datetime(ds)
    return (date.month >= 9) & (date.month <=11)

# La lógica fallaba, tuve que corregir. 
def is_winter(ds): 
    date = pd.to_datetime(ds)
    return (date.month == 12) | (date.month <=2)

# A esta función la hice de una forma distinta para que ande bien.
def is_weekend(ds):     
    return ds.dayofweek in (5, 6)

In [ ]:
# agregamos al set de entrenamiento
train_prophet['is_spring'] = train_prophet['ds'].apply(is_spring)
train_prophet['is_summer'] = train_prophet['ds'].apply(is_summer)
train_prophet['is_autumn'] = train_prophet['ds'].apply(is_autumn)
train_prophet['is_winter'] = train_prophet['ds'].apply(is_winter)
train_prophet['is_weekend'] = train_prophet['ds'].apply(is_weekend)
train_prophet['is_weekday'] = ~train_prophet['ds'].apply(is_weekend) 

In [ ]:
# is_winter & is_weekend no tienen valores True...porque? RESUELTO. 
# train_prophet[train_prophet["is_weekday"]==True][0:5]
train_prophet.shape

In [ ]:
# agregamos al set de testeo
test_prophet_2['is_spring'] = test_prophet_2['ds'].apply(is_spring)
test_prophet_2['is_summer'] = test_prophet_2['ds'].apply(is_summer)
test_prophet_2['is_autumn'] = test_prophet_2['ds'].apply(is_autumn)
test_prophet_2['is_winter'] = test_prophet_2['ds'].apply(is_winter)
test_prophet_2['is_weekend'] = test_prophet_2['ds'].apply(is_weekend)
test_prophet_2['is_weekday'] = ~test_prophet_2['ds'].apply(is_weekend)

In [ ]:
# test_prophet_2[test_prophet_2["is_spring"]==False][0:100]
test_prophet_2.info()

Instanciamos la clase con parámetros configurados

In [ ]:
prophet = Prophet(
    daily_seasonality = False,
    weekly_seasonality = False,
    yearly_seasonality = False
)

# Estacionalidades personalizadas para tener en cuenta las variaciones condicionales
# Tendencias extremas en extremas temporadas
prophet.add_seasonality(name='yearly', period=365.25, fourier_order = 10)
prophet.add_seasonality(name='weekly_spring', 
                        period=7,
                        fourier_order = 5,
                        condition_name='is_spring')
prophet.add_seasonality(name='weekly_summer', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_summer')
prophet.add_seasonality(name='weekly_autumn', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_autumn')
prophet.add_seasonality(name='weekly_winter', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_winter')
prophet.add_seasonality(name='daily_spring',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_spring')
prophet.add_seasonality(name='daily_summer',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_summer')
prophet.add_seasonality(name='daily_autumn',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_autumn')
prophet.add_seasonality(name='daily_winter',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_winter')
prophet.add_seasonality(name='daily_weekend',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_weekend')
prophet.add_seasonality(name='daily_weekday',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_weekday')

# Feriados/días festivos
# prophet.add_country_holidays(country_name = 'AR')

# fitting el modelo
prophet.fit(train_prophet);

#parte del dataframe en el que queremos hacer la prediccion
future = test_prophet_2.drop(['y'], axis=1)

# Prediciendo valores
forecast = prophet.predict(future)

pd.plotting.register_matplotlib_converters()

#graficando el componente de estacionalidad encontrado
_ = prophet.plot_components(forecast)

In [ ]:
forecast.tail()

## Guardo los datos obtenidos en un archivo .csv

In [ ]:
df_y = pd.DataFrame(forecast)
df_y.rename(columns={'ds':'DateTime', 'y':'Demanda en [MW]', 'yhat': 'Demanda pronosticada en [MW]'}, inplace = True)
df_y.set_index('DateTime')

In [ ]:
df_y.to_csv('forecasting_prophet.csv', columns=['Demanda pronosticada en [MW]'], encoding='utf-8')

Definimos función MAPE: error de porcentaje absoluto medio

In [ ]:
def mape(y_true, y_pred):
    """Error de porcentaje absoluto medio"""
    
    # conversión a vectores numpy
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # Porcentaje de error
    pe = (y_true - y_pred) / y_true
    
    # valor absolutos
    ape = np.abs(pe)
    
    # Cuantificación del rendimiento en un solo nº
    mape = np.mean(ape)
    
    return f'{mape*100:.2f}%'

Graficamos los resultados obtenidos: Curva de test y curva de valores predecidos

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_prophet.ds, y=test_prophet.y,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat,
                         mode='lines', 
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.update_layout(title='Prophet Forecast of Hourly Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# quantify accuracy
print(f'MAPE for Prophet\'s predictions: {mape(test_prophet.y, forecast.yhat[0:8809])}')

<b><i>El error es de 0% respecto al proyecto</i></b>

## Visualización de la primer y última semana de pronostico

In [ ]:
test_prophet_2[:168].index

In [ ]:
# Longitud de intervalo
interval = 24*7

# Necesitamos adaptar al intervalo las variables a usarse, 
# dado que la predicción se hizo por intervalos de 24*365
x_true, y_true = test_prophet_2.iloc[:interval].ds, test_prophet_2.iloc[:interval].y
x_pred, y_pred = forecast.iloc[:interval].ds, forecast.iloc[:interval].yhat

# Grafica
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode = 'lines',
                         name = 'Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode = 'lines',
                         name = 'Test - Prediction'))
# Ajustes varios sobre la grafica
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Prophet: Pronóstico de las primeras {interval} horas de Demanda',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# Eficacia 
print(f'MAPE para el intervalo en las primeras {interval} horas: {mape(y_true, y_pred)}')

<b><i>Error menor en 0.01% respecto al proyecto</i></b>

In [ ]:
# Longitud de intervalo
interval = -24*7

# Necesitamos adaptar al intervalo las variables a usarse, 
# dado que la predicción se hizo por intervalos de 24*365
x_true, y_true = test_prophet.iloc[interval:].ds, test_prophet.iloc[interval:].y
x_pred, y_pred = forecast[:-12407].iloc[interval:].ds, forecast[:-12407].iloc[interval:].yhat

# Grafica
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode = 'lines',
                         name = 'Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode = 'lines',
                         name = 'Test - Prediction'))
# Ajustes varios sobre la grafica
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Prophet: Pronóstico de las últimas {abs(interval)} horas de Demanda',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# Eficacia 
print(f'MAPE para el intervalo en las primeras {abs(interval)} horas: {mape(y_true, y_pred)}')

<b><i>Error menor en 0.02% respecto al proyecto</i></b>